# Make an interactive RGB composite
https://github.com/Unidata/unidata-python-workshop/blob/master/notebooks/Satellite_Data/GOES_Interactive_Plot.ipynb

In [1]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from IPython.display import display
from ipywidgets import DatePicker, Dropdown, interact, fixed, Select, FloatRangeSlider
from matplotlib import patheffects
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np
from scipy import interpolate
from siphon.catalog import TDSCatalog

In [2]:
def open_dataset(date, channel, idx, region):
    """
    Open and return a netCDF Dataset object for a given date, channel, and image index
    of GOES-16 data from THREDDS test server.
    """
    cat = TDSCatalog('http://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/'
             '{}/Channel{:02d}/{}/catalog.xml'.format(region, channel, date))
    dataset_name = list(cat.datasets)[idx]
    dataset = cat.datasets[dataset_name]                       
    ds = Dataset(dataset.access_urls['OPENDAP'])
    return ds

In [3]:
def plot_GOES16_channel(date, idx, channel, region):
    """
    Get and plot a GOES 16 data band from the ABI.
    """
    ds = open_dataset(date, channel, idx, region)
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]
    z = ds.variables['Sectorized_CMI'][:]
    proj_var = ds.variables[ds.variables['Sectorized_CMI'].grid_mapping]

    # Create a Globe specifying a spherical earth with the correct radius
    globe = ccrs.Globe(ellipse='sphere', semimajor_axis=proj_var.semi_major,
                       semiminor_axis=proj_var.semi_minor)

    proj = ccrs.LambertConformal(central_longitude=proj_var.longitude_of_central_meridian,
                                 central_latitude=proj_var.latitude_of_projection_origin,
                                 standard_parallels=[proj_var.standard_parallel],
                                 globe=globe)


    # Set up a feature for the state/province lines. Tell cartopy not to fill in the polygons
    state_boundaries = cfeat.NaturalEarthFeature(category='cultural',
                                                 name='admin_1_states_provinces_lakes',
                                                 scale='50m', facecolor='none')

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.coastlines(resolution='50m', color='black')
    ax.add_feature(state_boundaries, linestyle=':', edgecolor='black')
    ax.add_feature(cfeat.BORDERS, linewidth='2', edgecolor='black')

    for im in ax.images:
        im.remove()
    im = ax.imshow(z, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')
    timestamp = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
    # Add text (aligned to the right); save the returned object so we can manipulate it.
    text_time = ax.text(0.99, 0.01, timestamp.strftime('%d %B %Y %H%MZ'),
                   horizontalalignment='right', transform=ax.transAxes,
                   color='white', fontsize='x-large', weight='bold')

    text_channel = ax.text(0.5, 0.94, 'Channel {}'.format(channel),
                   horizontalalignment='center', transform=ax.transAxes,
                   color='white', fontsize='large', weight='bold')
    
    text_experimental = ax.text(0.5, 0.97, 'Experimental GOES-16 Data',
               horizontalalignment='center', transform=ax.transAxes,
               color='white', fontsize='large', weight='bold')

    # Make the text stand out even better using matplotlib's path effects
    # Make the text stand out even better using matplotlib's path effects
    outline_effect = [patheffects.withStroke(linewidth=2, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_channel.set_path_effects(outline_effect)
    text_experimental.set_path_effects(outline_effect)
    display(fig)
    plt.close()

In [4]:
channel_list = {'1 - Blue Band': 1,
                '2 - Red Band': 2,
                '3 - Veggie Band': 3,
                '4 - Cirrus Band': 4,
                '5 - Snow/Ice Band:': 5,
                '6 - Cloud Particle Size Band': 6,
                '7 - Shortwave Window Band': 7,
                '8 - Upper-Level Tropo. WV Band': 8,
                '9 - Mid-Level Tropo. WV Band': 9,
                '10 - Low-Level WV Band': 10,
                '11 - Cloud-Top Phase Band': 11,
                '12 - Ozone Band': 12,
                '13 - Clean IR Longwave Band': 13,
                '14 - IR Longwave Band': 14,
                '15 - Dirty Longwave Band': 15,
                '16 - CO2 Longwave IR': 16}

region = Select(
    options=['Mesoscale-1', 'Mesoscale-2', 'CONUS'],
    #value='Mesoscale 1',
    description='Region:',
)

channel = Dropdown(
    options=channel_list,
    value=9,
    description='Channel:',
)

date = datetime.strftime(datetime.utcnow(), '%Y%m%d')
interact(plot_GOES16_channel, date=fixed(date), idx=fixed(-2), 
         channel=channel, region=region)

interactive(children=(Dropdown(description='Channel:', index=8, options={'1 - Blue Band': 1, '2 - Red Band': 2, '3 - Veggie Band': 3, '4 - Cirrus Band': 4, '5 - Snow/Ice Band:': 5, '6 - Cloud Particle Size Band': 6, '7 - Shortwave Window Band': 7, '8 - Upper-Level Tropo. WV Band': 8, '9 - Mid-Level Tropo. WV Band': 9, '10 - Low-Level WV Band': 10, '11 - Cloud-Top Phase Band': 11, '12 - Ozone Band': 12, '13 - Clean IR Longwave Band': 13, '14 - IR Longwave Band': 14, '15 - Dirty Longwave Band': 15, '16 - CO2 Longwave IR': 16}, value=9), Select(description='Region:', options=('Mesoscale-1', 'Mesoscale-2', 'CONUS'), value='Mesoscale-1'), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_GOES16_channel>